# MLflow with Amazon SageMaker

This sample notebook shows how to deploy ML models registered in MLflow to Amazon SageMaker Endpoint.

This notebook works well with the `Data Science 3.0` kernel on a SageMaker Studio `ml.t3.medium` instance.

Here is a list of packages that are used in this notebook.

```
!pip freeze | grep -E "mlflow|boto3|urllib3|scikit-learn|sagemaker"
-------------------------------------------------------------------
boto3==1.28.63
mlflow==2.6.0
sagemaker==2.194.0
sagemaker-data-insights @ https://files.pythonhosted.org/packages/70/8b/7c964508afe1dc3535422df8383c022c762c1f1254acb68b29d26b33fe30/sagemaker_data_insights-0.3.3-py3-none-any.whl
sagemaker-datawrangler @ https://files.pythonhosted.org/packages/6a/29/6d3da0518cbe72647b164bbdee23f4df3936cf5691fff9b29dc8714115ff/sagemaker_datawrangler-0.4.3-py3-none-any.whl
sagemaker-scikit-learn-extension==2.5.0
sagemaker-studio-analytics-extension==0.0.20
sagemaker-studio-sparkmagic-lib==0.1.4
scikit-learn==1.3.1
scikit-learn-intelex==2021.20221004.171507
urllib3==2.0.6
```

### Set up environments

In [ ]:
!python --version

Python 3.10.6


In [ ]:
!pip install -U sagemaker mlflow==2.6.0 scikit-learn==1.3.1

In [ ]:
!pip freeze | grep -E "mlflow|boto3|urllib3|scikit-learn|sagemaker"

boto3==1.28.63
mlflow==2.6.0
sagemaker==2.194.0
sagemaker-data-insights @ https://files.pythonhosted.org/packages/70/8b/7c964508afe1dc3535422df8383c022c762c1f1254acb68b29d26b33fe30/sagemaker_data_insights-0.3.3-py3-none-any.whl
sagemaker-datawrangler @ https://files.pythonhosted.org/packages/6a/29/6d3da0518cbe72647b164bbdee23f4df3936cf5691fff9b29dc8714115ff/sagemaker_datawrangler-0.4.3-py3-none-any.whl
sagemaker-scikit-learn-extension==2.5.0
sagemaker-studio-analytics-extension==0.0.20
sagemaker-studio-sparkmagic-lib==0.1.4
scikit-learn==1.3.1
scikit-learn-intelex==2021.20221004.171507
urllib3==2.0.6


### Set MLflow Tracking URI¶

In [ ]:
MLFLOW_TRACKING_URI = "<MLflow Tracking URI>" # "http://ec2-192-168-0-1.compute-1.amazonaws.com:5000"

In [ ]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

### Make Predictions with the stored Model

In [ ]:
logged_model = 'runs:<Run ID>/model' # 'runs:/9c6bbde5cd2041cfab006f143130f797/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
predictions = loaded_model.predict(X_test)
print(predictions)

2023/10/23 05:55:36 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 1.26.0, required: numpy==1.26.1)
 - psutil (current: 5.9.0, required: psutil==5.9.6)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[226.83103356 133.06536317 106.816427   144.19174514 112.33193859
 114.6142718   85.86407134 103.4670468  223.05704266  96.29232487
 121.35298745 136.34504633 101.63430644  91.86921416 208.98175501
 178.23790024 258.01554956  89.88840908 176.81103395 121.29993359
 168.24772002  88.56835009 206.2750657  127.26343568 217.93116615
  95.81491159  77.57568969 167.80270092 121.88011086  94.94957898
  89.65418724  80.45384154 129.82726442  87.15359573 177.82565659
 198.47918338 174.9246423   80.11173384 263.54381454 125.29878665
 169.55055357  86.1489538  248.45871157 173.46414303 220.23449431
 133.14258575 121.91847456  99.38080433 106.10998897 120.91077318
 102.93417805 146.7487253   75.22910757 162.98302521 101.01144296
 143.08073036 204.1223701   90.4316589  136.10699756  97.11165675
  95.03763941 100.63006363 218.92265849 232.62916403 214.99984777
 183.42818541  97.18773832  84.13705769 127.89108644 106.59108026
 187.48947093 246.16022797 195.75472146  92.54101476 161.66918529
  89.86874

In [ ]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 9c6bbde5cd2041cfab006f143130f797

# Deployment

### (Option 1) Using `mlflow.sagemaker` library

##### Prequisites

We need to build Docker image to be used for a base image in SageMaker Endpoint, and push it to your AWS ECR.

The docker image can be build and pushed by runing following commands:

```
aws configure set region us-east-1
mlflow sagemaker build-and-push-container 
```

**WARNING**: To run `mlflow sagemaker build-and-push-container`, docker should be running on the machine.
For more information, see [MLflow Command-Line Interface](https://www.mlflow.org/docs/latest/cli.html#mlflow-sagemaker).


In [ ]:
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
region = sess.boto_region_name

In [ ]:
from mlflow.deployments import get_deploy_client
from mlflow.sagemaker import SageMakerDeploymentClient
import pandas as pd

mlflow_client = get_deploy_client(f"sagemaker:/{region}")

In [ ]:
instance_type = 'ml.m5.large'

config = {
    "execution_role_arn": role,
    "region_name": region,
    "bucket_name": bucket,
    "archive": False,
    "instance_type": instance_type,
    "instance_count": 1,
    "synchronous": True,
    "timeout_seconds": 1800
}

config

In [ ]:
endpoint_name = "diabets-regression"

mlflow_client.create_deployment(endpoint_name,
                         model_uri=logged_model,
                         flavor="python_function",
                         config=config)

In [ ]:
mlflow_client.predict(endpoint_name, X_test)

### (Option 2) Using SageMaker Endpoint

In [ ]:
import os

model_data = './sk_model'
os.makedirs(model_data, exist_ok=True)

model_uri = logged_model
model = mlflow.sklearn.load_model(model_uri, model_data)

In [ ]:
%%sh

cd sk_model
tar -czvf model.tar.gz model/

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = sess.default_bucket()

region, bucket

In [ ]:
!aws s3 cp sk_model/model.tar.gz s3://{bucket}/mlflow-sagemaker/diabetes-regression/model.tar.gz

In [ ]:
!cp sk_model/model/requirements.txt code/

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

FRAMEWORK_VERSION = '1.2-1'

model_data = f"s3://{bucket}/mlflow-sagemaker/diabetes-regression/model.tar.gz"
sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    entry_point="inference.py",
    source_dir="code",
    framework_version=FRAMEWORK_VERSION
)

In [ ]:

from sagemaker.utils import name_from_base

endpoint_name = name_from_base("diabetes-regression")

sklearn_predictor = sklearn_model.deploy(instance_type="ml.m5.large",
                                         initial_instance_count=1,
                                         endpoint_name=endpoint_name)

In [ ]:
print(f"Serializer: {sklearn_predictor.serializer}")
print(f"Deserializer: {sklearn_predictor.deserializer}")
print(f"ContentType: {sklearn_predictor.content_type}")
print(f"Accept: {sklearn_predictor.accept}")

In [ ]:
sklearn_predictor.predict(X_test)

# Clean Up

### (Option 1) Using `mlflow.sagemaker` library

In [ ]:
endpoint_name = "diabets-regression"

mlflow_client.delete_deployment(endpoint_name)

### (Option 2) Using SageMaker Endpoint

In [ ]:
sklearn_predictor.delete_model()
sklearn_predictor.delete_endpoint()